---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def degree_distribution(G):    
    degrees = G.degree()
    degree_values = sorted(set(degrees.values()))
    histogram = [list(degrees.values()).count(i)/float(nx.number_of_nodes(G)) for i in degree_values]
    return histogram


In [4]:
def graph_identification():
    
    # Your Code Here
    algorithm = []
    for i in range(len(P1_Graphs)):
        clustering = nx.average_clustering(P1_Graphs[i])
        shortest_path = nx.average_shortest_path_length(P1_Graphs[i]) 
        degree = degree_distribution(P1_Graphs[i])
        if len(degree)>10:
            algorithm.append('PA')
        elif clustering < 0.1:
            algorithm.append('SW_H')
        else:
            algorithm.append('SW_L')
    return algorithm        
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [5]:
G = nx.read_gpickle('email_prediction.txt')

#print(nx.info(G))

### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [6]:
#import libraries

from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler

In [7]:
#I create a function that label none,0,1 on management salary
def is_management(node):
    #access management salary value
    management_salary = node[1]['ManagementSalary'] 
    if management_salary == 1:
        return 1
    elif management_salary == 0:
        return 0
    else:
        return None

In [8]:
def salary_predictions():
     # Your Code Here
    
    # okay first we create dataframe
    df = pd.DataFrame(index=G.nodes())
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(nx.degree(G))
    df['degree_centrality'] = pd.Series(nx.degree_centrality(G))
    df['closeness'] = pd.Series(nx.closeness_centrality(G, normalized=True))
    df['betweenness'] = pd.Series(nx.betweenness_centrality(G, normalized=True))
    df['page_rank'] = pd.Series(nx.pagerank(G))
    df['is_management'] = pd.Series(is_management(node) for node in G.nodes(data=True))
    
    #next we create df_train -> is_management=0/1 and df_test -> is_management = NaN
    df_train = df[~pd.isnull(df['is_management'])]
    df_test  = df[pd.isnull(df['is_management'])]
    
    #next we separate X and y
    X_train = df_train.drop('is_management', axis=1)
    y_train = df_train['is_management']
    
    X_test  = df_test.drop('is_management', axis=1)
    
    #standarization X train dan X test
    scaler = MinMaxScaler().fit(X_train)
    X_train_scl = scaler.transform(X_train)
    X_test_scl  = scaler.transform(X_test)
    
    # okay let's build ml model
    model = MLPClassifier(hidden_layer_sizes=[10,5],
                          alpha=5, random_state=0,
                          solver='lbfgs', verbose=0).fit(X_train_scl,y_train)
    probabilities = model.predict_proba(X_test_scl)[:,1] # we just need the probability
    
    return pd.Series(probabilities, index=X_test.index)

salary_predictions()

1       0.128439
2       0.586261
5       0.979859
8       0.133702
14      0.304625
18      0.208766
27      0.269586
30      0.336144
31      0.169191
34      0.143513
37      0.096221
40      0.195036
45      0.162267
54      0.249954
55      0.252905
60      0.176908
62      0.992842
65      0.551284
77      0.078478
79      0.132108
97      0.099812
101     0.057517
103     0.214342
108     0.122930
113     0.454489
122     0.044350
141     0.396296
142     0.863703
144     0.072049
145     0.419966
          ...   
913     0.045734
914     0.078520
915     0.002087
918     0.144304
923     0.035175
926     0.090079
931     0.055969
934     0.013933
939     0.011147
944     0.002353
945     0.039608
947     0.162715
950     0.098173
951     0.038313
953     0.027345
959     0.002922
962     0.002440
963     0.139886
968     0.110279
969     0.104672
974     0.095061
984     0.012668
987     0.110375
989     0.113364
991     0.116944
992     0.003956
994     0.003077
996     0.0024

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [9]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [10]:
#example of accesing node
G.node[1]

{'Department': 1, 'ManagementSalary': nan}

In [12]:
def new_connections_predictions():
    
    # Your Code Here
    
    # first we create dataframe
    df2 = future_connections
    df2['preferential_attachment'] = [i[2] for i in nx.preferential_attachment(G, future_connections.index)]
    df2['common_neighbors'] = [len(list(nx.common_neighbors(G, x[0], x[1]))) for x in future_connections.index]
    df2['jaccard_coefficient'] = [i[2] for i in nx.jaccard_coefficient(G, future_connections.index)]
    df2['resource_allocation'] = [i[2] for i in nx.resource_allocation_index(G, future_connections.index)]
    df2['adamic_adar'] = [i[2] for i in nx.adamic_adar_index(G, future_connections.index)]
    
    # community assignment
    for n in G.nodes():
        G.node[n]['community'] = G.node[n]['Department']
    df2['cn_soundarajan_hopcroft'] = [i[2] for i in nx.cn_soundarajan_hopcroft(G, future_connections.index)]
    df2['ra_soundarajan_hopcroft'] = [i[2] for i in nx.ra_index_soundarajan_hopcroft(G, future_connections.index)]
    
    #let's build df train -> future connection 0,1 and test -> future connection NaN
    df_train = df2[~pd.isnull(df2['Future Connection'])]
    df_test  = df2[pd.isnull(df2['Future Connection'])]
    
    X_train = df_train.drop('Future Connection', axis=1)
    y_train = df_train['Future Connection']
    X_test  = df_test.drop('Future Connection', axis=1)
    
    scaler = MinMaxScaler().fit(X_train)
    X_train_scl = scaler.transform(X_train)
    X_test_scl  = scaler.transform(X_test)
    
    model = RandomForestClassifier(n_estimators=100, max_depth=10).fit(X_train_scl, y_train)
    probabilities = model.predict_proba(X_test_scl)[:,1]
    return pd.Series(probabilities, X_test.index)
new_connections_predictions()

(107, 348)    0.026686
(542, 751)    0.012652
(20, 426)     0.571752
(50, 989)     0.012606
(942, 986)    0.012714
(324, 857)    0.012615
(13, 710)     0.127624
(19, 271)     0.146779
(319, 878)    0.012579
(659, 707)    0.012652
(49, 843)     0.012670
(208, 893)    0.012652
(377, 469)    0.013092
(405, 999)    0.016522
(129, 740)    0.012796
(292, 618)    0.017940
(239, 689)    0.012481
(359, 373)    0.017933
(53, 523)     0.383242
(276, 984)    0.012384
(202, 997)    0.012618
(604, 619)    0.040801
(270, 911)    0.012544
(261, 481)    0.061291
(200, 450)    0.988885
(213, 634)    0.012627
(644, 735)    0.041768
(346, 553)    0.012444
(521, 738)    0.012052
(422, 953)    0.014304
                ...   
(672, 848)    0.012544
(28, 127)     0.981239
(202, 661)    0.012444
(54, 195)     0.999814
(295, 864)    0.012625
(814, 936)    0.012627
(839, 874)    0.012714
(139, 843)    0.012615
(461, 544)    0.013688
(68, 487)     0.013260
(622, 932)    0.012583
(504, 936)    0.017266
(479, 528) 